In [168]:
import duckdb as ddb
import pandas as pd

con = ddb.connect(database=":memory:")

con.sql("""
CREATE TABLE program_beslut AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-beslut.csv';
CREATE TABLE program_kommun AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-diarie_kommun.csv';

CREATE TABLE kurser_beslut AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-beslut.csv';
CREATE TABLE kurser_kommun AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-diarie_kommun.csv';
""")

In [169]:
con.sql("""select * from kurser_beslut""").describe()

┌─────────┬───────────────┬──────────────────┬─────────┬─────────────────────────────────┬─────────────────────────────────────────────────┬───────────────────┬───────────────────┬────────────────────┬──────────────────────────┬────────────────────┬──────────────┬───────────┬────────────────────┬────────────────┐
│  aggr   │ Diarienummer  │ Ansökningsomgång │ Beslut  │      Utbildningsanordnare       │                 Utbildningsnamn                 │ Utbildningsområde │     YH-poäng      │ Beviljade platser  │ Beviljade platser totalt │       Start        │     Län      │  Kommun   │   Antal kommuner   │ Flera kommuner │
│ varchar │    varchar    │      double      │ varchar │             varchar             │                     varchar                     │      varchar      │      double       │       double       │          double          │       double       │   varchar    │  varchar  │       double       │    varchar     │
├─────────┼───────────────┼──────────────────┼─────────

In [170]:
def get_utbildningsanordnare(con):
    rel = con.sql("""
    select distinct Utbildningsanordnare
    from kurser_beslut
    order by Utbildningsanordnare
    """)
    return [i[0] for i in rel.fetchall()]

get_utbildningsanordnare(con)

['ABF Göteborg Vuxenutbildning AB',
 'AGSTU AB',
 'ALEA Kompetenshöjning AB',
 'AVT AB',
 'Ai Högskolan Sverige AB',
 'Almost There AB',
 'Amsac AB',
 'Angered folkhögskola',
 'Arboga kommun, Vuxenutbildningen',
 'Arvika Kommun',
 'Bergsskolan Kompetensutveckling AB',
 'Blue Peak AB',
 'Borås Stad - Yrkeshögskolan i Borås',
 'Båstads kommun Akademi Båstad',
 'Cady training utbildningar AB',
 'Campus Ljungby AB',
 'Campus Nyköping',
 'Campus Nynäshamn',
 'Campus Varberg/Varbergs kommun',
 'Changemaker Educations AB',
 'Chas Academy AB',
 'Dans i Göteborg AB',
 'Digitala lyftet i Sverige AB',
 'Dille Gård AB',
 'EBT Academy AB',
 'EC Utbildning AB Göteborg',
 'EC Utbildning AB Malmö',
 'EC Utbildning AB Stockholm',
 'EC Utbildning AB Örebro',
 'East Sweden Business School AB',
 'EdUP Group AB',
 'Edugrade AB',
 'Elevera Education Group',
 'Enköpings kommun, Yrkeshögskolan i Enköping',
 'Equine Academy Sweden AB',
 'Eriksmåla Ridklubb',
 'Falkenbergs kommun',
 'Fastighetsakademin Sverige 

In [171]:
def create_df_utbildningsomrade_beviljade(con, utbildningsanordnare: list[str] | None = None, year=2024):
    rel = con.sql("""
    select
        Utbildningsområde,
        count(*) as Beviljade
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
        and Beslut = true
    group by Utbildningsområde
    order by Beviljade desc, Utbildningsområde
    ;""",
    params={"year": year, "utb": utbildningsanordnare})
    return rel.df().set_index(["Utbildningsområde"])

create_df_utbildningsomrade_beviljade(con)

,Beviljade
Utbildningsområde,
"Ekonomi, administration och försäljning",171
Data/IT,111
Teknik och tillverkning,106
Samhällsbyggnad och byggteknik,93
Hälso- och sjukvård samt socialt arbete,81
"Hotell, restaurang och turism",19
"Lantbruk, djurvård, trädgård, skog och fiske",18
Pedagogik och undervisning,15
Journalistik och information,11


In [172]:
create_df_utbildningsomrade_beviljade(con, ["Stiftelsen Stockholms Tekniska Institut"])

,Beviljade
Utbildningsområde,
Data/IT,7
Samhällsbyggnad och byggteknik,6


In [173]:
def create_df_utbildningsomrade_avslag(con, utbildningsanordnare: list[str] | None = None, year=2024):
    rel = con.sql("""
    select
        Utbildningsområde,
        count(*) as Avslag
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
        and Beslut = false
    group by Utbildningsområde
    order by Avslag desc, Utbildningsområde
    ;""",
    params={"year": year, "utb": utbildningsanordnare})
    return rel.df().set_index("Utbildningsområde")

create_df_utbildningsomrade_avslag(con)

,Avslag
Utbildningsområde,
"Ekonomi, administration och försäljning",112
Data/IT,84
Hälso- och sjukvård samt socialt arbete,53
Teknik och tillverkning,51
Samhällsbyggnad och byggteknik,41
"Lantbruk, djurvård, trädgård, skog och fiske",24
Pedagogik och undervisning,18
Säkerhetstjänster,11
"Hotell, restaurang och turism",9


In [174]:
create_df_utbildningsomrade_avslag(con, ["Stiftelsen Stockholms Tekniska Institut"])

,Avslag
Utbildningsområde,
Data/IT,1
Samhällsbyggnad och byggteknik,1


In [179]:
def create_df_utbildningsomrade_beslut(con, beslut=True, utbildningsanordnare: list[str] | None = None, year=2024):
    rel = con.sql("""
    select
        Utbildningsområde,
        count(*) as Beslut
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
        and Beslut = $beslut
    group by Utbildningsområde
    order by Beslut desc, Utbildningsområde
    ;""",
    params={"beslut": beslut, "year": year, "utb": utbildningsanordnare})
    df_rel: pd.Dataframe = rel.df().set_index("Utbildningsområde")
    return df_rel.rename(columns={"Beslut": "Beviljade" if beslut else "Avslag"})

create_df_utbildningsomrade_beslut(con, beslut=False)

,Avslag
Utbildningsområde,
"Ekonomi, administration och försäljning",112
Data/IT,84
Hälso- och sjukvård samt socialt arbete,53
Teknik och tillverkning,51
Samhällsbyggnad och byggteknik,41
"Lantbruk, djurvård, trädgård, skog och fiske",24
Pedagogik och undervisning,18
Säkerhetstjänster,11
"Hotell, restaurang och turism",9


In [176]:
def get_kurser_antal(con):
    rel = con.sql("""
    select
        (select count(*) from kurser_beslut) as "Kurs-ansökningar",
        (select count(*) from kurser_kommun) as "Kurs-ansökningar, alla kommuner"
    """)
    return rel.fetchone()

get_kurser_antal(con)

(1090, 1202)

In [177]:
def get_utbildningsomraden(con):
    rel = con.sql("""
    select distinct Utbildningsområde
    from kurser_beslut
    order by Utbildningsområde
    """)
    return [i[0] for i in rel.fetchall()]

get_utbildningsomraden(con)

['Data/IT',
 'Ekonomi, administration och försäljning',
 'Friskvård och kroppsvård',
 'Hotell, restaurang och turism',
 'Hälso- och sjukvård samt socialt arbete',
 'Journalistik och information',
 'Juridik',
 'Kultur, media och design',
 'Lantbruk, djurvård, trädgård, skog och fiske',
 'Pedagogik och undervisning',
 'Samhällsbyggnad och byggteknik',
 'Säkerhetstjänster',
 'Teknik och tillverkning',
 'Transporttjänster']

In [178]:
def create_df_kurser_beviljade(con, utbildningsomrade: list[str] | None = None):
    rel = con.sql("""
    select
        Utbildningsanordnare,
        -- Utbildningsområde,
        count(*) as Ansökningar,
        sum(case when Beslut = true then 1 else 0 end)::integer as Beviljade,
        round(100.00 * sum(case when Beslut = true then 1 else 0 end) / count(*),1) as "Beviljade %",
    from kurser_beslut
    where ( $areas IS NULL OR Utbildningsområde = ANY($areas) )
    group by Utbildningsanordnare, Utbildningsområde
    order by "Beviljade %" desc, Ansökningar desc, Utbildningsanordnare
    """,
    params={"areas": utbildningsomrade})
    return rel.df().set_index("Utbildningsanordnare")

create_df_kurser_beviljade(con, ["Data/IT"]).head(20)

,Ansökningar,Beviljade,Beviljade %
Utbildningsanordnare,,,
IT-Högskolan Sverige AB,11,11,100.0
Nackademin AB,9,9,100.0
YH Akademin AB,8,8,100.0
Handelsakademin i Göteborg AB,6,6,100.0
EC Utbildning AB Göteborg,4,4,100.0
AGSTU AB,2,2,100.0
Folkuniversitetet - Stiftelsen Kursverksamheten vid Uppsala universitetet,2,2,100.0
Hermods AB Örebro,2,2,100.0
Hyper Island Program AB - Karlskrona,2,2,100.0
